# Creazione dataset

## generazione immagini

In [1]:
!pip install Pillow numpy pandas torch torchvision

  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 3.8 MB/s eta 0:00:00a 0:00:01
Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl (11.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 3.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
dataset_dir = f"./digit_dataset"

In [4]:
from PIL import Image, ImageDraw, ImageFont
import os, numpy as np, pandas as pd, torch
import random
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms # Import the transforms module

predisporre all'interno della cartella `./fonts` i font da cui verranno generati i caratteri

In [26]:
# Parametri
#chars_to_generate = np.array(list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789,;.:!?'()[]{}<>/\\@#$€£%&~aèéìòù-+°"))

img_size = (28, 28)

fonts_path = [os.path.join(f"./fonts", file) for file in os.listdir(f"./fonts") if file.endswith(".ttf")]
fonts_name = [os.path.splitext(os.path.basename(file))[0] for file in fonts_path]

font_size = 20

colors = np.array(["white", "black", "red", "blue", "green", "yellow"])
text_colors = colors[:1]
background_colors = colors[1:2]

In [27]:
# Crea la directory principale
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

In [28]:
import os
from PIL import Image, ImageDraw, ImageFont

# Assume necessary variables are defined elsewhere, for example:
# dataset_dir = "generated_dataset"
# fonts_path = ["/path/to/your/font1.ttf", "/path/to/your/font2.otf"] # Replace with actual font paths
# chars_to_generate = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"
# img_size = (64, 64) # Target image size (width, height)
# text_colors = [(0, 0, 0), (255, 255, 255)] # Black and White
# background_colors = [(255, 255, 255), (0, 0, 0)] # White and Black


def get_ink_bbox(image, background_color, tolerance=5):
	"""
	Finds the tight bounding box of non-background pixels in an image.

	Args:
		image: PIL Image object.
		background_color: The background color tuple (R, G, B).
		tolerance: Allows for a small difference in color to be considered background.

	Returns:
		A tuple (left, top, right, bottom) representing the ink bounding box,
		or None if the image is all background within the tolerance.
	"""
	width, height = image.size
	left, top = width, height
	right, bottom = -1, -1

	# Ensure background_color is in the same mode as the image for comparison
	try:
		# Create a 1x1 image with the background color in the target image mode
		bg_color_check = Image.new(image.mode, (1, 1), color=background_color).getpixel((0, 0))
	except Exception:
		# print(f"Warning: Could not match background color mode for ink bbox detection. Using provided background_color directly.")
		bg_color_check = background_color

	pixels = image.load()

	def is_background(pixel, bg_color, tol):
		# Compare pixel color to background color within tolerance
		if isinstance(pixel, tuple) and isinstance(bg_color, tuple):
			 if len(pixel) == len(bg_color):
				 return all(abs(p - bg) <= tol for p, bg in zip(pixel, bg_color))
			 elif len(pixel) == 4 and len(bg_color) == 3: # Compare RGBA pixel to RGB background (ignore alpha of pixel)
				  return all(abs(pixel[i] - bg_color[i]) <= tol for i in range(3))
		elif isinstance(pixel, int) and isinstance(bg_color, int): # Handle grayscale or paletted images
			 return abs(pixel - bg_color) <= tol
		return pixel == bg_color # Fallback for exact match if types are unexpected


	for y in range(height):
		for x in range(width):
			if not is_background(pixels[x, y], bg_color_check, tolerance):
				left = min(left, x)
				top = min(top, y)
				right = max(right, x)
				bottom = max(bottom, y)

	if right == -1: # Image is all background within tolerance
		return None

	return (left, top, right, bottom)

def generate_character_image(font_path, char, text_color, background_color, img_size, dataset_dir, char_idx, top_space=0, bottom_space=0):
	"""
	Generates an image for a single character with specified spacing.

	Args:
		font_path: Path to the font file.
		char: The character to render.
		text_color: The color of the text (R, G, B).
		background_color: The background color (R, G, B).
		img_size: The target size of the output image (width, height).
		dataset_dir: The base directory for the dataset.
		char_idx: The index of the character (for directory naming).
		top_space: The number of pixels to leave as empty space at the top.
		bottom_space: The number of pixels to leave as empty space at the bottom.

	Returns:
		The relative path to the saved image if successful, None otherwise.
	"""
	font_name = os.path.splitext(os.path.basename(font_path))[0]
	char_dir = os.path.join(dataset_dir, str(char_idx))
	if not os.path.exists(char_dir):
		os.makedirs(char_dir)

	# Use a large initial font size for accurate ink bbox measurement
	initial_font_size_for_bbox = 100 # Increased initial size for better accuracy
	# Create a temporary image much larger than the initial font size
	# This ensures the entire glyph is captured, even for characters
	# with large ascenders/descenders or unusual placements.
	temp_img_size = (initial_font_size_for_bbox * 4, initial_font_size_for_bbox * 4)

	# Create a temporary image and draw object
	# Use 'RGB' mode for consistency in pixel analysis
	temp_img = Image.new('RGB', temp_img_size, color=background_color)
	temp_draw = ImageDraw.Draw(temp_img)

	# Load the font with the initial large size
	initial_font = ImageFont.truetype(font_path, initial_font_size_for_bbox)

	# Get the layout bounding box of the character with the initial font
	# This is used to position the character on the temporary image
	# relative to where Pillow thinks its origin is.
	try:
		# Use textbbox for newer Pillow versions
		temp_layout_bbox = temp_draw.textbbox((0, 0), char, font=initial_font)
	except AttributeError:
			# Fallback to textsize for older Pillow versions
			temp_text_width, temp_text_height = temp_draw.textsize(char, font=initial_font)
			temp_layout_bbox = (0, 0, temp_text_width, temp_text_height) # Approximate bbox

	temp_text_width = temp_layout_bbox[2] - temp_layout_bbox[0]
	temp_text_height = temp_layout_bbox[3] - temp_layout_bbox[1]

	# Calculate drawing position on the temporary image to roughly center the layout bbox
	# Adding a buffer to ensure ink is not clipped at edges
	buffer = initial_font_size_for_bbox // 2
	draw_x_temp = (temp_img_size[0] - temp_text_width) / 2 - temp_layout_bbox[0] + buffer
	draw_y_temp = (temp_img_size[1] - temp_text_height) / 2 - temp_layout_bbox[1] + buffer


	# Draw the character on the temporary image
	temp_draw.text((draw_x_temp, draw_y_temp), char, fill=text_color, font=initial_font)

	# Find the tight ink bounding box on the temporary image
	# Use a small tolerance to account for anti-aliasing
	ink_bbox = get_ink_bbox(temp_img, background_color, tolerance=10) # Increased tolerance slightly

	if ink_bbox is None:
			print(f"Warning: Could not find ink for character '{char}' with font '{font_name}'. Skipping this combination.")
			return None

	ink_left, ink_top, ink_right, ink_bottom = ink_bbox
	ink_width = ink_right - ink_left + 1 # Add 1 because bbox is inclusive
	ink_height = ink_bottom - ink_top + 1 # Add 1

	if ink_width <= 0 or ink_height <= 0:
			print(f"Warning: Ink bounding box has zero or negative dimension for character '{char}' with font '{font_name}'. Skipping this combination.")
			return None

	# Calculate the effective height for scaling, considering top and bottom space
	effective_height = img_size[1] - top_space - bottom_space
	if effective_height <= 0:
			print(f"Error: Effective height for character '{char}' with font '{font_name}' is zero or negative after applying top/bottom space. Skipping.")
			return None

	# Calculate scaling factor based on ink bbox and the effective image size
	width_ratio = img_size[0] / ink_width
	height_ratio = effective_height / ink_height
	scale_factor = min(width_ratio, height_ratio)

	# Calculate the optimal font size
	optimal_font_size = int(initial_font_size_for_bbox * scale_factor)

	# Ensure optimal font size is reasonable
	if optimal_font_size <= 0:
		optimal_font_size = 1
		print(f"Warning: Calculated optimal font size for character '{char}' with font '{font_name}' is too small. Setting to 1.")
	# Prevent excessively large font sizes that could cause memory issues or errors
	elif optimal_font_size > 200: # Increased arbitrary cap
			optimal_font_size = 200
			print(f"Warning: Calculated optimal font size for character '{char}' with font '{font_name}' is excessively large. Capping at {optimal_font_size}.")


	# Create the final image with the correct size and background
	img = Image.new('RGB', img_size, color=background_color)
	draw = ImageDraw.Draw(img)

	# Create the font instance with the optimal size
	if optimal_font_size <= 0:
			print(f"Error: Optimal font size for character '{char}' with font '{font_name}' is invalid ({optimal_font_size}). Skipping drawing.")
			return None
	font = ImageFont.truetype(font_path, optimal_font_size)

	# Calculate the expected ink bbox at the optimal size by scaling
	# The ink bbox coordinates on the temporary image are relative to its top-left corner.
	# We need their position relative to the draw origin on the temporary image.
	ink_bbox_relative_to_draw_temp = (ink_left - draw_x_temp, ink_top - draw_y_temp, ink_right - draw_x_temp, ink_bottom - draw_y_temp)

	# Scale these relative coordinates to the optimal font size scale
	scaled_ink_left = ink_bbox_relative_to_draw_temp[0] * scale_factor
	scaled_ink_top = ink_bbox_relative_to_draw_temp[1] * scale_factor
	scaled_ink_right = ink_bbox_relative_to_draw_temp[2] * scale_factor
	scaled_ink_bottom = ink_bbox_relative_to_draw_temp[3] * scale_factor

	# Calculate the drawing position on the final image to center the scaled ink bbox
	# within the effective height (img_size[1] - top_space - bottom_space) starting at top_space.
	# The draw.text position (x, y) is the top-left of the text's layout bounding box.
	# We want the center of the scaled ink bbox to be at the horizontal center of the image
	# and vertically centered within the effective height area.

	center_x_img = img_size[0] / 2
	center_y_effective_area = top_space + effective_height / 2

	# The center of the scaled ink bbox, relative to the final image's draw origin (0,0)
	center_x_ink_scaled = (scaled_ink_left + scaled_ink_right) / 2
	center_y_ink_scaled = (scaled_ink_top + scaled_ink_bottom) / 2

	# The required draw position (x, y) on the final image is where the layout bbox origin
	# should be placed so that the scaled ink bbox is centered as desired.
	draw_x = center_x_img - center_x_ink_scaled
	draw_y = center_y_effective_area - center_y_ink_scaled


	# Draw the character on the final image
	draw.text((draw_x, draw_y), char, fill=text_color, font=font)

	# Save the image
	img_filename = f"{font_name}_{background_color[0]}_{background_color[1]}_{background_color[2]}_{text_color[0]}_{text_color[1]}_{text_color[2]}_{top_space}_{bottom_space}.png"
	img_path = os.path.join(char_dir, img_filename)
	img = img.convert('L').point(lambda x: 0 if x < 128 else 255, mode='1')
	img.save(img_path)

	# Return relative path from dataset_dir
	return os.path.relpath(img_path, dataset_dir)

# --- Main script execution ---
dataset_file = os.path.join(dataset_dir, "dataset.txt")

# Define top and bottom space (example values)
top_space_pixels = 5
bottom_space_pixels = 5

In [29]:
# top and bottom spaces
chars_with_spaces = [
	("ABCDEFGHIJKLMNOPQRSTUVWXYZbdfhklt0123456789!?()[]{}/\\@#$€£%&", range(0, 2), range(0,6)),
	("aceimnorsuvwxz:aèéìòù", range(0, 16), range(0,6)),
	("gjpqy;", range(0,16), range(0,2)),
	(",", range(10, 24), range(0,2)),
	(".", range(10, 24), range(0,6)),
	(":°", range(10, 20), range(0,6)),
	("~-+", range(0, 20), range(0,12)),
	("'°", range(0, 2), range(0,24))
]

In [30]:
dataset_file = os.path.join(dataset_dir, "dataset.txt")

with open(dataset_file, "w", encoding="utf-8") as f:
	for path in fonts_path[:1]:
		font_name = os.path.splitext(os.path.basename(path))[0]
		print(f"Processing font: {font_name}")

		for idx1, (char_list, top_range, bottom_range) in enumerate(chars_with_spaces):
			for idx2, char in enumerate(char_list):
				print(f"  Generating images for character: '{char}' (Index: {idx1} - {idx2})")

				for text_color in text_colors:
					for background_color in background_colors:
						if text_color == background_color:
							continue

						for t_sp in top_range:
							for b_sp in bottom_range:
								relative_img_path = generate_character_image(
									font_path=fonts_path[0],
									char=char,
									text_color=text_color,
									background_color=background_color,
									img_size=img_size,
									dataset_dir=dataset_dir,
									char_idx=str(idx1)+"_"+str(idx2),
									top_space=t_sp,
									bottom_space=b_sp
								)

								if relative_img_path:
									# Write to dataset.txt
									f.write(f"{relative_img_path},{char}\n")

print("Image generation complete.")


Processing font: PTS55F
  Generating images for character: 'A' (Index: 0 - 0)
  Generating images for character: 'B' (Index: 0 - 1)
  Generating images for character: 'C' (Index: 0 - 2)
  Generating images for character: 'D' (Index: 0 - 3)
  Generating images for character: 'E' (Index: 0 - 4)
  Generating images for character: 'F' (Index: 0 - 5)
  Generating images for character: 'G' (Index: 0 - 6)
  Generating images for character: 'H' (Index: 0 - 7)
  Generating images for character: 'I' (Index: 0 - 8)
  Generating images for character: 'J' (Index: 0 - 9)
  Generating images for character: 'K' (Index: 0 - 10)
  Generating images for character: 'L' (Index: 0 - 11)
  Generating images for character: 'M' (Index: 0 - 12)
  Generating images for character: 'N' (Index: 0 - 13)
  Generating images for character: 'O' (Index: 0 - 14)
  Generating images for character: 'P' (Index: 0 - 15)
  Generating images for character: 'Q' (Index: 0 - 16)
  Generating images for character: 'R' (Index: 0 - 

dividiamo il file `dataset.txt` in due file `train.txt` e `test.txt`

In [52]:
train_file = os.path.join(dataset_dir, "train.txt")
test_file = os.path.join(dataset_dir, "test.txt")

# Leggi tutte le righe
with open(dataset_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Mischia le righe casualmente
random.shuffle(lines)

# Dividi 80% train e 20% test
split_idx = int(0.8 * len(lines))
train_lines = lines[:split_idx]
test_lines = lines[split_idx:]

# Scrivi i file train.txt e test.txt
with open(train_file, "w", encoding="utf-8") as f:
    f.writelines(train_lines)

with open(test_file, "w", encoding="utf-8") as f:
    f.writelines(test_lines)

creare un file `classes.txt` che conterrà in ogni riga i nomi dei file dataset.

Quindi avremo questo contenuto:
```
train.txt
test.txt
```

**TODO**: da capire se dobbiamo mettere il validation

## pytorch dataset

In [56]:
class DigitDataset(Dataset):
	def __init__(self, imgs_path, txt_path, transform=None):
		self.imgs_path = imgs_path
		self.images = np.loadtxt(txt_path, dtype=str, delimiter=',')
		self.transform = transform

	def __getitem__(self, index):
		# carica il path e indice dell'immagine
		f, c = self.images[index]

		im = Image.open(f'{self.imgs_path}/{f}')

		if self.transform is not None:
			im = self.transform(im)

		return {
			"image": im,
			"label": c
		}

	def __len__(self):
		return len(self.images)

In [57]:
train_path = f'{dataset_dir}/train.txt'

dataset = DigitDataset(imgs_path=dataset_dir, txt_path=train_path, transform=transforms.ToTensor())

In [58]:
sample = dataset[0]
print(sample['image'].shape)
print(sample['label'])

torch.Size([3, 28, 28])
S


## data normalization

In [60]:
x_pixels_count = int(sample['image'].shape[1])
y_pixels_count = int(sample['image'].shape[2])
x_pixels_count, y_pixels_count

(28, 28)

media

In [61]:
m = np.zeros(3)
for sample in dataset:
  m += sample['image'].sum(1).sum(1).numpy()

m = m / (len(dataset) * x_pixels_count * y_pixels_count)

In [62]:
"media", m

('media', array([0.49894239, 0.41612808, 0.33276246]))

varianza

In [63]:
s = np.zeros(3)
for sample in dataset:
  s += ((sample['image'] - torch.Tensor(m).view(3,1,1)) ** 2).sum(1).sum(1).numpy()

s = np.sqrt(s / (len(dataset) * x_pixels_count * y_pixels_count))

In [64]:
"varianza", s

('varianza', array([0.49364248, 0.44286803, 0.46521507]))

normalizziamo

In [65]:
transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize(m, s),
    transforms.Lambda(lambda x: x.view(-1))
])

dataset = DigitDataset(imgs_path=dataset_dir, txt_path=train_path, transform=transform)
print(dataset[0]['image'].shape)
print(dataset[0]['label'])

torch.Size([2352])
S
